In [1]:
%matplotlib inline
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm
from keras.models import load_model
from keras.layers import GlobalAveragePooling2D, Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation
from keras.preprocessing.image import ImageDataGenerator

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


In [2]:
tf.compat.v1.enable_eager_execution(
    config=None, device_policy=None, execution_mode=None
)

In [3]:
from helper import get_class_names, get_train_data, get_test_data, plot_images
from helper import plot_model, predict_classes, visualize_errors

In [4]:
matplotlib.style.use('ggplot')

In [5]:
class_names = get_class_names()
print(class_names)

Decoding file: data/batches.meta
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [6]:
num_classes = len(class_names)
print(num_classes)

10


In [7]:
# Hight and width of the images
IMAGE_SIZE = 32
# 3 channels, Red, Green and Blue
CHANNELS = 3
# Number of epochs
NUM_EPOCH = 110
# learning rate
LEARN_RATE = 1.0e-4

In [8]:
images_train, labels_train, class_train = get_train_data()

Decoding file: data/data_batch_1
Decoding file: data/data_batch_2
Decoding file: data/data_batch_3
Decoding file: data/data_batch_4
Decoding file: data/data_batch_5


In [9]:
images_test, labels_test, class_test = get_test_data()

Decoding file: data/test_batch


In [10]:
print("Training set size:\t",len(images_train))
print("Testing set size:\t",len(images_test))

Training set size:	 50000
Testing set size:	 10000


In [11]:
def pure_cnn_model():
    
    model = Sequential()
    
    model.add(Conv2D(96, (3, 3), activation='relu', padding = 'same', input_shape=(IMAGE_SIZE,IMAGE_SIZE,CHANNELS)))    
    model.add(Dropout(0.2))
    
    model.add(Conv2D(96, (3, 3), activation='relu', padding = 'same'))  
    model.add(Conv2D(96, (3, 3), activation='relu', padding = 'same', strides = 2))    
    model.add(Dropout(0.5))
    
    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))    
    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same', strides = 2))    
    model.add(Dropout(0.5))    
    
    model.add(Conv2D(192, (3, 3), padding = 'same'))
    model.add(Activation('relu'))
    model.add(Conv2D(192, (1, 1),padding='valid'))
    model.add(Activation('relu'))
    model.add(Conv2D(10, (1, 1), padding='valid'))

    model.add(GlobalAveragePooling2D())
    
    model.add(Activation('softmax'))

    model.summary()
    
    return model

In [12]:
model = pure_cnn_model()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 96)        2688      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 96)        83040     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 96)        83040     
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 192)       166080    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 192)      

In [13]:
checkpoint = ModelCheckpoint('best_model_improved.h5', 
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only= True,
                             mode='auto')  

In [14]:
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=LEARN_RATE),
              metrics = ['accuracy']) 

In [15]:
model_details = model.fit(images_train, class_train,
                    batch_size = 128,
                    epochs = NUM_EPOCH, # number of iterations
                    validation_data= (images_test, class_test),
                    callbacks=[checkpoint],
                    verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/110
50000/50000 [==============================] - 61s 1ms/step - loss: 1.9836 - accuracy: 0.2394 - val_loss: 1.7870 - val_accuracy: 0.3341
Epoch 2/110
50000/50000 [==============================] - 58s 1ms/step - loss: 1.6825 - accuracy: 0.3686 - val_loss: 1.6742 - val_accuracy: 0.3853
Epoch 3/110
50000/50000 [==============================] - 58s 1ms/step - loss: 1.5722 - accuracy: 0.4170 - val_loss: 1.4860 - val_accuracy: 0.4593
Epoch 4/110
50000/50000 [==============================] - 61s 1ms/step - loss: 1.4976 - accuracy: 0.4486 - val_loss: 1.4560 - val_accuracy: 0.4625
Epoch 5/110
50000/50000 [==============================] - 60s 1ms/step - loss: 1.4422 - accuracy: 0.4705 - val_loss: 1.3735 - val_accuracy: 0.5011
Epoch 6/110
50000/50000 [==============================] - 1079s 22ms/step - loss: 1.3849 - accuracy: 0.4938 - val_loss: 1.3202 - val_accuracy: 0.5248
Epoch 7/110
50000/50000 [==============================] - 

Epoch 56/110
50000/50000 [==============================] - 57s 1ms/step - loss: 0.6135 - accuracy: 0.7840 - val_loss: 0.6815 - val_accuracy: 0.7637
Epoch 57/110
50000/50000 [==============================] - 57s 1ms/step - loss: 0.6073 - accuracy: 0.7883 - val_loss: 0.6548 - val_accuracy: 0.7708
Epoch 58/110
50000/50000 [==============================] - 57s 1ms/step - loss: 0.6048 - accuracy: 0.7893 - val_loss: 0.6564 - val_accuracy: 0.7712
Epoch 59/110
50000/50000 [==============================] - 57s 1ms/step - loss: 0.5977 - accuracy: 0.7902 - val_loss: 0.6543 - val_accuracy: 0.7716
Epoch 60/110
50000/50000 [==============================] - 57s 1ms/step - loss: 0.5910 - accuracy: 0.7923 - val_loss: 0.6780 - val_accuracy: 0.7637
Epoch 61/110
50000/50000 [==============================] - 57s 1ms/step - loss: 0.5822 - accuracy: 0.7960 - val_loss: 0.6529 - val_accuracy: 0.7714
Epoch 62/110
50000/50000 [==============================] - 57s 1ms/step - loss: 0.5724 - accuracy: 0.7998

In [16]:
scores = model.evaluate(images_test, class_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 81.98%


In [17]:
correct, labels_pred = predict_classes(model, images_test, labels_test)

In [18]:
num_images = len(correct)
print("Accuracy: %.2f%%" % ((sum(correct)*100)/num_images))

Accuracy: 81.98%


In [25]:
ans = []
for i in range(10000):
    ans.append(class_names[labels_pred[i]])

In [26]:
ans

['dog',
 'ship',
 'ship',
 'airplane',
 'frog',
 'frog',
 'automobile',
 'frog',
 'cat',
 'automobile',
 'airplane',
 'truck',
 'dog',
 'horse',
 'truck',
 'ship',
 'dog',
 'horse',
 'ship',
 'frog',
 'automobile',
 'airplane',
 'deer',
 'truck',
 'deer',
 'bird',
 'deer',
 'airplane',
 'truck',
 'frog',
 'frog',
 'dog',
 'deer',
 'cat',
 'truck',
 'automobile',
 'deer',
 'truck',
 'truck',
 'dog',
 'deer',
 'frog',
 'dog',
 'frog',
 'airplane',
 'truck',
 'cat',
 'truck',
 'horse',
 'frog',
 'truck',
 'ship',
 'horse',
 'cat',
 'ship',
 'ship',
 'horse',
 'cat',
 'cat',
 'dog',
 'horse',
 'dog',
 'frog',
 'cat',
 'frog',
 'bird',
 'automobile',
 'bird',
 'cat',
 'horse',
 'bird',
 'cat',
 'ship',
 'ship',
 'airplane',
 'bird',
 'truck',
 'cat',
 'dog',
 'ship',
 'ship',
 'automobile',
 'automobile',
 'horse',
 'bird',
 'dog',
 'airplane',
 'airplane',
 'ship',
 'truck',
 'airplane',
 'cat',
 'ship',
 'frog',
 'deer',
 'frog',
 'frog',
 'airplane',
 'airplane',
 'horse',
 'deer',
 'dog

In [28]:
np.savetxt("cifar10CNN.csv",ans, fmt = '%s')